Libraries

In [104]:
import numpy as np
from scipy.linalg import expm

In [105]:

from qiskit.quantum_info import SparsePauliOp, Operator
from qiskit_algorithms import TimeEvolutionProblem
from qiskit_algorithms import TrotterQRTE
from qiskit.quantum_info import Statevector
from qiskit.synthesis import SuzukiTrotter, LieTrotter
from qiskit.primitives import StatevectorEstimator

# for fake backend
from qiskit.providers.fake_provider import GenericBackendV2
from qiskit import transpile
from qiskit import QuantumCircuit

Convertor of text to binary

In [112]:
#def text_to_binary(text):
    #return ' '.join(format(ord(char), '08b') for char in text)

def text_to_binary(text):
    return ''.join(format(ord(char), '08b') for char in text)

Theory

In [362]:
def A_matrix(N):
    if N < 2:
        raise ValueError("Path graph must have at least 2 nodes.")
    
    A = np.zeros((N, N), dtype=int)
    for i in range(N-2):
        A[i, i+1] = 1
        A[i+1, i] = 1

    return A

def D_matrix(N):
    if N < 2:
        raise ValueError("Path graph must have at least 2 nodes.")
    
    D = np.zeros((N, N), dtype=int)
    for i in range(N-1):
        if i == 0 or i == N-1:
            D[i, i] = 1
        else:
            D[i, i] = 2
    
    return D

def L_matrix(D, A):
    L = D-A
    return L

def A_cycle(N):
    if N < 3:
        raise ValueError("Cycle graph must have at least 3 nodes.")
    
    A = np.zeros((N, N), dtype=int)
    for i in range(N):
        A[i, (i+1) % N] = 1  # Connect to next node (with wrap-around)
        A[(i+1) % N, i] = 1  # Connect back
    
    return A

def D_cycle(N):
    if N < 3:
        raise ValueError("Cycle graph must have at least 3 nodes.")
    
    # In a cycle graph, each node has degree 2
    D = 2 * np.eye(N, dtype=int)
    
    return D

def L_cycle(D, A):
    L = D - A
    return L

def U_0(A, t):
    evolution_0 = expm(-1j*t*A)
    return evolution_0

def U_1(L, t):
    evolution_1 = expm(-1j*t*L)
    return evolution_1

def initial_state(N, position):
    if N <= 0:
        raise ValueError("N must be a positive integer.")
    
    state = np.zeros(N, dtype=complex)
    state[position] = 1
    return state

def CTQW(message,t):
    num_bits = len(message)

    if num_bits <= 0:
        raise ValueError("Number of bits must be a positive integer.")
    
    state = initial_state(15, 0)
    #evolution_0 = U_0(A_cycle(num_bits), t)
    #evolution_1 = U_1(L_cycle(D_cycle(num_bits), A_cycle(num_bits)), t)
    evolution_0 = U_0(A_matrix(15), t)
    evolution_1 = U_1(L_matrix(D_matrix(15), A_matrix(15)), t)

    for bit in range(num_bits):
        if message[bit] == '0':
            new_state = evolution_0 @ state
        else:
            new_state = evolution_1 @ state
        state = new_state
    return state

def prob_dist(state):
    prob_dist = np.abs(state)**2
    for i in range(len(prob_dist)):
        prob_dist[i] = prob_dist[i] * 2* 10**4

    #norm = np.sum(prob_dist)
    #norm_prob_dist = prob_dist / norm
    return prob_dist

def get_binary(prob_dist):
    binary_list = [format(int(p), '012b') for p in prob_dist]
    
    # Concatenate all binary values into one string
    hash_value = ''.join(binary_list)
    
    return hash_value

def get_hash(prob_dist):
    list = [format(int(p * 4095), '03x') for p in prob_dist]
    hash_value = ''.join(list)
    
    return hash_value

In [357]:
#text = text_to_binary("Hi")
#print(text)
text = '110'

CTQW_run = CTQW(text, 1.2)
print("Probability distribution:", prob_dist(CTQW_run))
hash_value = get_hash(prob_dist(CTQW_run))
print("Hash value:", hash_value)

Probability distribution: [  141.29811927 19858.70188073     0.        ]
Hash value: 8d4374d8dda8000


Trotterization

In [309]:
def matrix_to_pauli(matrix):
    operator = Operator(matrix)
    pauli_list = SparsePauliOp.from_operator(operator)
    return pauli_list

def trotterization(initial_state, final_time, pauli_list, type, repetitions):

    problem = TimeEvolutionProblem(pauli_list, initial_state=initial_state, time=final_time)

    trotter = TrotterQRTE(product_formula=type, num_timesteps=repetitions, estimator= StatevectorEstimator())
    result = trotter.evolve(problem)

    return result

def trotter_circuit(num_qubits, backend, initial_state, final_time, hamiltonian, type, repetitions, optimization_level):
    pauli_list = matrix_to_pauli(hamiltonian)
    qc = QuantumCircuit(num_qubits)
    
    trotter_circuit = trotterization(initial_state, final_time, pauli_list, type, repetitions).evolved_state

    qc.compose(trotter_circuit, inplace=True)
    qc.measure_all()

    transpiled_circuit = transpile(qc, backend, optimization_level)

    return transpiled_circuit

def calculate_trotter_prob_distribution(initial_state, final_time, hamiltonian, type, repetitions):
    # Trotterize the time evolution
    pauli_list = matrix_to_pauli(hamiltonian)
    trotter_circuit = trotterization(initial_state, final_time, pauli_list, type, repetitions).evolved_state
    print(trotter_circuit)
    trotter_result = Statevector(trotter_circuit)
    trotter_prob_distribution = np.abs(trotter_result)**2
    
    return trotter_prob_distribution

def calculate_trotter_state(initial_state, final_time, hamiltonian, type, repetitions):
    pauli_list = matrix_to_pauli(hamiltonian)
    trotter_circuit = trotterization(initial_state, final_time, pauli_list, type, repetitions).evolved_state
    #print(trotter_circuit)
    trotter_result = Statevector(trotter_circuit)
    return trotter_result

def trotter_circuit_real_backend(num_qubits, backend, initial_state, final_time, hamiltonian, type, repetitions):
    pauli_list = matrix_to_pauli(hamiltonian)
    qc = QuantumCircuit(num_qubits)
    
    trotter_circuit = trotterization(initial_state, final_time, pauli_list, type, repetitions).evolved_state

    qc.compose(trotter_circuit, inplace=True)
    qc.measure_all()

    return qc   

def draw_circuit(result, reps):
    return result.decompose().draw(output='mpl', fold=reps, idle_wires=False, scale=0.8, reverse_bits=True)

Algorithm on simulator

In [379]:
def CTQW_simulator(message,t):
    num_bits = len(message)

    if num_bits <= 0:
        raise ValueError("Number of bits must be a positive integer.")
    
    #state = initial_state(num_bits, 0)
    num_qubits = 3
    state = Statevector.from_label('0'* num_qubits)  # Initial state |000...0>
    type = LieTrotter()  # or SuzukiTrotter()
    repetitions = 30
    #A = A_matrix(num_bits)
    #L = L_matrix(D_matrix(num_bits), A)
    size = 2**num_qubits
    A = A_cycle(size)
    L = L_cycle(D_cycle(size), A)
    #A = A_matrix(size)
    #L = L_matrix(D_matrix(size), A)

    qc = QuantumCircuit(num_qubits)
    for bit in range(num_bits):
        if message[bit] == '0':
            new_state = calculate_trotter_state(state, t, A, type, repetitions)
        else:
            new_state = calculate_trotter_state(state, t, L, type, repetitions)
        state = new_state
    return state

#text = text_to_binary("ow")
#print(text)
#text = '1101'
# random 12 bits
text = np.random.randint(2, size=12).astype(str).tolist()
print(text)

t = 2
CTQW_run = CTQW_simulator(text, t)
#print("Probability distribution:", prob_dist(CTQW_run))
hash_value = get_hash(prob_dist(CTQW_run))
print("Hash value:", hash_value)

['0', '1', '1', '0', '1', '0', '1', '0', '1', '1', '1', '1']
Hash value: 205298f2d6dcf39737401cfd185f826e223e168d1b3ab80c0


Analysis

In [380]:
# collision test
def random_binary_string(length):
    return ''.join(np.random.choice(['0', '1'], size=length))

def collision_test(num_strings, length):
    seen_hashes = set()
    seen_strings = set()
    collisions = 0

    for _ in range(num_strings):
        # Generate a new unique random string
        while True:
            random_string = random_binary_string(length)
            if random_string not in seen_strings:
                seen_strings.add(random_string)
                break
        
        time = np.pi
        CTQW_run = CTQW_simulator(random_string, time)
        hash_binary_value = get_binary(prob_dist(CTQW_run))
        print("Hash value:", hash_binary_value)
        if hash_binary_value in seen_hashes:
            collisions += 1
        else:
            seen_hashes.add(hash_binary_value)

    return collisions


# mean number of bits changed
def mean_changed_bits(num_experiments, length):
    total_changed_bits = 0

    for _ in range(num_experiments):
        random_string = random_binary_string(length)
        time = 4
        CTQW_run = CTQW_simulator(random_string, time)
        prob_dist_result = prob_dist(CTQW_run)
        original_hash_value = get_binary(prob_dist_result)

        total_changed_bits = 0
        for _ in range(num_experiments):
        # randomly change some bits in the random string
            changed_string = ''.join(np.random.choice(['0', '1'], size=length))
            CTQW_run_changed = CTQW(changed_string, time)
            changed_prob_dist = prob_dist(CTQW_run_changed)
            changed_hash_value = get_binary(changed_prob_dist)

            changed_bits = 0
            for bit in range(length):
                if original_hash_value[bit] != changed_hash_value[bit]:
                    changed_bits += 1
            total_changed_bits += changed_bits

    return total_changed_bits / num_experiments

# mean probability of change per bit P
def mean_changed_probability(num_experiments, length):
    mean = mean_changed_bits(num_experiments, length)
    total_hash_length = 180
    p = (mean / total_hash_length)*100
    return p  # Return as a percentage

# Standard variance of the changed probability(
def std_variance_changed_probability(num_experiments, length):
    probabilities = []

    for _ in range(num_experiments):
        random_string = random_binary_string(length)
        time = 4
        CTQW_run = CTQW(random_string, time)
        prob_dist_result = prob_dist(CTQW_run)
        changed_bits = np.sum(prob_dist_result > 0.01)  # Threshold to consider a bit changed
        p = (changed_bits / 180) * 100  # Assuming each bit contributes to a hash of length 4096
        probabilities.append(p)

    return np.std(probabilities)  # Return the standard deviation of the mean change per bit probability
    
"""
# standard deviation of the mean change per bit probability
def std_deviation_mean_changed_probability(num_experiments, length):
    random_string = random_binary_string(length)
    time = 4
    CTQW_run = CTQW(random_string, time)
    prob_dist_result = prob_dist(CTQW_run)
    original_hash_value = get_binary(prob_dist_result)

    total_changed_bits = 0
    changed_bits_list = []
    for _ in range(num_experiments):
    # randomly change some bits in the random string
        changed_string = ''.join(np.random.choice(['0', '1'], size=length))
        CTQW_run_changed = CTQW(changed_string, time)
        changed_hash_value = prob_dist(CTQW_run_changed)

        changed_bits = 0
        for bit in range(length):
            if original_hash_value[bit] != changed_hash_value[bit]:
                changed_bits += 1
        
        changed_bits_list.append(changed_bits)
        total_changed_bits += changed_bits

    mean_changed_bits_ = total_changed_bits / num_experiments
\   
    difference = changed_bits_list - mean_changed_bits_
    delta_beta = np.sqrt(np.sum(changed_bits_list - mean_changed_bits_)**2/(num_experiments - 1))
    return delta_beta
"""


"\n# standard deviation of the mean change per bit probability\ndef std_deviation_mean_changed_probability(num_experiments, length):\n    random_string = random_binary_string(length)\n    time = 4\n    CTQW_run = CTQW(random_string, time)\n    prob_dist_result = prob_dist(CTQW_run)\n    original_hash_value = get_binary(prob_dist_result)\n\n    total_changed_bits = 0\n    changed_bits_list = []\n    for _ in range(num_experiments):\n    # randomly change some bits in the random string\n        changed_string = ''.join(np.random.choice(['0', '1'], size=length))\n        CTQW_run_changed = CTQW(changed_string, time)\n        changed_hash_value = prob_dist(CTQW_run_changed)\n\n        changed_bits = 0\n        for bit in range(length):\n            if original_hash_value[bit] != changed_hash_value[bit]:\n                changed_bits += 1\n\n        changed_bits_list.append(changed_bits)\n        total_changed_bits += changed_bits\n\n    mean_changed_bits_ = total_changed_bits / num_experim

Run tests

In [382]:
# collision test run
collision_test(10, 5)

# mean changed bits run
#mean_changed = mean_changed_bits(100, 12)
#print("Mean number of changed bits: ", mean_changed)


#mean of added bit number of zeroes

#mean changed probability
#p = mean_changed_probability(100, 12)
#print("Mean changed probability p: ", p, " %")

# standard variance of the changed probability



Hash value: 11100111101110001101101011000010010010010101011001000110000001001101101011000010010010010101011001
Hash value: 11100111101110011001011001000000000001001011111101000110000001011001011001000000000001001011111101
Hash value: 1010010010101000000010110000000000011010000000111111110110100100000001011000000000001101000000011
Hash value: 1010111000000110001010000000100100101011100010101111010110011011000101000000010010010101110001010
Hash value: 1010111000000110001010000000100100101011100010101111010110011011000101000000010010010101110001010
Hash value: 1010111000001011100010100000100100100110001010001111010110011101110001010000010010010011000101000
Hash value: 11100111101110001101101011000010010010010101011001000110000001001101101011000010010010010101011001
Hash value: 1010010010101000101111100000100100101000101111101111110110100100010111110000010010010100010111110
Hash value: 1010010010100111100100010000001010101010010101001111110110100011110010001000000101010101001010100
Hash va

2

In [301]:
print(len('16509f0000905e10310110315e109000009f'))

36
